In [1]:
import pickle
from layers import *
import layers

2024-04-25 23:03:11.687814: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 23:03:11.688040: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 23:03:11.690402: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 23:03:11.719910: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 23:03:12.300659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [6]:
tf.keras.Layer == tf.keras.layers.Layer

True

In [4]:
x = xin = Input((3,)) # input could be a multidim vector (e.g., (5,3) you have have 5 embeddings with dims of 3)

# y = BatchNorm(x, 0.1)
# y = SkipConn(y, 0.1, 5, x) # skip conn from input
# y = Attn(y, 0.1, 3)
# y = Dense(y, 0.1, 5)
# y = Conv(y, 0.1, 3)
# z = BatchNorm(y, 0.1)
# y = Dense(z, 0.1, 4)
# y = SkipConn(y, 0.1, 4, z)

# x = Attn(x, 0.1, 3)
x = Dense(x, 0.1, 3)
y=x
model = y

model([
    [1,2, 4]
])

# model.layer.activation

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.17954886, 0.01934875, 0.4621845 ]], dtype=float32)>

In [5]:
x.layer.get_build_config()

{'input_shape': (1, 3)}

In [19]:
model([
    [1,2, 4]
])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.05546863,  3.6908638 , -0.11977742]], dtype=float32)>

In [17]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('model.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [18]:
model2([
    [1,2, 4]
])

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[ 0.36200842, -0.2318269 , -0.2111532 ],
        [ 0.35188818, -0.23154292, -0.21405986],
        [ 0.33908635, -0.23107196, -0.21765728]]], dtype=float32)>